# ETL3 - SCRIPT DE PIPELINE DE DADOS

##### Script responsável por buscar os dados coletados do CENSO 2019 e CENSO 2020 que estão no Cloud Storage, transformar eles para atender as necessidades do projeto e, por fim, salvá-los na zona de outro do Cloud Storage em formato PARQUET

In [1]:
# Importa as bibliotecas necessárias
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f

In [2]:
# Obtem ou Cria uma Spark Session 
spark = SparkSession.builder.getOrCreate()

#### DEFINIÇÕES GERAIS

In [ ]:
# Define todas as colunas dos dados dos docentes que não serão utilizadas neste projeto
cols_docente_drop = ('NU_ANO_CENSO','ID_DOCENTE','NU_IDADE','IN_BAIXA_VISAO','IN_CEGUEIRA','IN_DEF_AUDITIVA','IN_DEF_FISICA',               'IN_DEF_INTELECTUAL','IN_SURDEZ', 'IN_SURDOCEGUEIRA', 'IN_DEF_MULTIPLA', 'IN_AUTISMO', 'IN_SUPERDOTACAO',
                     'TP_NORMAL_MAGISTERIO','TP_SITUACAO_CURSO_1','CO_AREA_CURSO_1','CO_CURSO_1','IN_LICENCIATURA_1','IN_COM_PEDAGOGICA_1','NU_ANO_INICIO_1','NU_ANO_CONCLUSAO_1','TP_TIPO_IES_1','CO_IES_1',
                     'TP_SITUACAO_CURSO_2','CO_AREA_CURSO_2','CO_CURSO_2','IN_LICENCIATURA_2','IN_COM_PEDAGOGICA_2','NU_ANO_INICIO_2','NU_ANO_CONCLUSAO_2','TP_TIPO_IES_2','CO_IES_2','TP_SITUACAO_CURSO_3','CO_AREA_CURSO_3',
                     'CO_CURSO_3','IN_LICENCIATURA_3','IN_COM_PEDAGOGICA_3','NU_ANO_INICIO_3','NU_ANO_CONCLUSAO_3','TP_TIPO_IES_3','CO_IES_3','IN_DISC_QUIMICA','IN_DISC_FISICA','IN_DISC_MATEMATICA','IN_DISC_BIOLOGIA',
                     'IN_DISC_CIENCIAS','IN_DISC_LINGUA_PORTUGUESA','IN_DISC_LINGUA_INGLES','IN_DISC_LINGUA_ESPANHOL','IN_DISC_LINGUA_FRANCES','IN_DISC_LINGUA_OUTRA','IN_DISC_LINGUA_INDIGENA','IN_DISC_ARTES',
                     'IN_DISC_EDUCACAO_FISICA','IN_DISC_HISTORIA', 'IN_DISC_GEOGRAFIA','IN_DISC_FILOSOFIA','IN_DISC_ENSINO_RELIGIOSO','IN_DISC_ESTUDOS_SOCIAIS','IN_DISC_SOCIOLOGIA','IN_DISC_EST_SOCIAIS_SOCIOLOGIA',
                     'IN_DISC_INFORMATICA_COMPUTACAO','IN_DISC_PROFISSIONALIZANTE','IN_DISC_ATENDIMENTO_ESPECIAIS','IN_DISC_DIVER_SOCIO_CULTURAL','IN_DISC_LIBRAS','IN_DISC_PEDAGOGICAS','IN_DISC_OUTRAS',                     'IN_ESPECIFICO_CRECHE', 'IN_ESPECIFICO_PRE_ESCOLA','IN_ESPECIFICO_ANOS_INICIAIS',
                     'IN_ESPECIFICO_ANOS_FINAIS','IN_ESPECIFICO_ENS_MEDIO','IN_ESPECIFICO_EJA','IN_ESPECIFICO_ED_ESPECIAL','IN_ESPECIFICO_ED_INDIGENA','IN_ESPECIFICO_CAMPO','IN_ESPECIFICO_AMBIENTAL','IN_ESPECIFICO_DIR_HUMANOS',
                     'IN_ESPECIFICO_DIV_SEXUAL','IN_ESPECIFICO_DIR_ADOLESC','IN_ESPECIFICO_AFRO','IN_ESPECIFICO_OUTROS','IN_ESPECIFICO_NENHUM','IN_REGULAR','IN_EJA','IN_PROFISSIONALIZANTE','CO_CURSO_EDUC_PROFISSIONAL',
                     'CO_REGIAO','CO_MESORREGIAO','CO_MICRORREGIAO','CO_UF','CO_MUNICIPIO','CO_DISTRITO','TP_DEPENDENCIA','TP_LOCALIZACAO','TP_CATEGORIA_ESCOLA_PRIVADA','IN_CONVENIADA_PP','TP_CONVENIO_PODER_PUBLICO',                     'IN_MANT_ESCOLA_PRIVADA_EMP','IN_MANT_ESCOLA_PRIVADA_ONG','IN_MANT_ESCOLA_PRIVADA_SIND',
                     'IN_MANT_ESCOLA_PRIVADA_SIST_S','IN_MANT_ESCOLA_PRIVADA_S_FINS','TP_REGULAMENTACAO','TP_LOCALIZACAO_DIFERENCIADA','IN_EDUCACAO_INDIGENA','CO_AREA_COMPL_PEDAGOGICA_1',
                     'CO_AREA_COMPL_PEDAGOGICA_2','CO_AREA_COMPL_PEDAGOGICA_3','IN_DISC_PORT_SEGUNDA_LINGUA','IN_DISC_ESTAGIO_SUPERVISIONADO','TP_TIPO_ATENDIMENTO_TURMA','TP_TIPO_LOCAL_TURMA', 'TP_MEDIACAO_DIDATICO_PEDAGO','IN_MANT_ESCOLA_PRIVADA_OSCIP','IN_MANT_ESCOLA_PRIV_ONG_OSCIP','TP_LOCAL_RESID_DIFERENCIADA','IN_COMPLEMENTACAO_PEDAGOGICA')

In [ ]:
# Define todas as colunas dos dados das matrículas que não serão utilizadas neste projeto
cols_matricula_drop = ('NU_IDADE','ID_ALUNO','NU_DURACAO_TURMA','NU_DUR_ATIV_COMP_MESMA_REDE','NU_DUR_ATIV_COMP_OUTRAS_REDES',                      'NU_DUR_AEE_MESMA_REDE','NU_DUR_AEE_OUTRAS_REDES','IN_CEGUEIRA','IN_BAIXA_VISAO','IN_SURDEZ',
                      'IN_DEF_AUDITIVA','IN_SURDOCEGUEIRA','IN_DEF_FISICA','IN_DEF_INTELECTUAL','IN_DEF_MULTIPLA','IN_AUTISMO',
                      'IN_SINDROME_ASPERGER','IN_SINDROME_RETT','IN_SUPERDOTACAO','IN_RECURSO_LEDOR','IN_RECURSO_TRANSCRICAO',
                      'IN_RECURSO_INTERPRETE','IN_RECURSO_LIBRAS','IN_RECURSO_LABIAL','IN_RECURSO_BRAILLE',                      'IN_RECURSO_AMPLIADA_16','IN_RECURSO_AMPLIADA_20','IN_RECURSO_AMPLIADA_24','IN_RECURSO_NENHUM',
                      'TP_INGRESSO_FEDERAIS','IN_REGULAR','IN_EJA','IN_PROFISSIONALIZANTE','CO_CURSO_EDUC_PROFISSIONAL',                      'TP_UNIFICADA','TP_TIPO_TURMA','CO_REGIAO','CO_MESORREGIAO','CO_MICRORREGIAO','CO_UF','CO_MUNICIPIO',
                      'CO_DISTRITO','TP_DEPENDENCIA','TP_LOCALIZACAO','TP_CATEGORIA_ESCOLA_PRIVADA','IN_CONVENIADA_PP',                      'TP_CONVENIO_PODER_PUBLICO','TP_CATEGORIA_ESCOLA_PRIVADA','IN_CONVENIADA_PP','TP_CONVENIO_PODER_PUBLICO',
                      'IN_MANT_ESCOLA_PRIVADA_EMP','IN_MANT_ESCOLA_PRIVADA_ONG','IN_MANT_ESCOLA_PRIVADA_SIND',                     'IN_MANT_ESCOLA_PRIVADA_SIST_S','IN_MANT_ESCOLA_PRIVADA_S_FINS','TP_REGULAMENTACAO',
                      'TP_LOCALIZACAO_DIFERENCIADA','IN_EDUCACAO_INDIGENA','NU_ANO_CENSO','IN_RECURSO_AMPLIADA_18',                       'IN_RECURSO_CD_AUDIO','IN_RECURSO_PROVA_PORTUGUES','IN_RECURSO_VIDEO_LIBRAS','IN_AEE_LIBRAS',
                       'IN_AEE_LINGUA_PORTUGUESA','IN_AEE_INFORMATICA_ACESSIVEL','IN_AEE_BRAILLE','IN_AEE_CAA',                       'IN_AEE_SOROBAN','IN_AEE_VIDA_AUTONOMA','IN_AEE_OPTICOS_NAO_OPTICOS','IN_AEE_ENRIQ_CURRICULAR',
                       'IN_AEE_DESEN_COGNITIVO','IN_AEE_MOBILIDADE','IN_TRANSP_BICICLETA','IN_TRANSP_MICRO_ONIBUS',                       'IN_TRANSP_ONIBUS','IN_TRANSP_TR_ANIMAL','IN_TRANSP_VANS_KOMBI','IN_TRANSP_OUTRO_VEICULO',
                       'IN_TRANSP_EMBAR_ATE5','IN_TRANSP_EMBAR_5A15','IN_TRANSP_EMBAR_15A35','IN_TRANSP_EMBAR_35','IN_MANT_ESCOLA_PRIVADA_OSCIP','IN_MANT_ESCOLA_PRIV_ONG_OSCIP','TP_MEDIACAO_DIDATICO_PEDAGO','TP_LOCAL_RESID_DIFERENCIADA','TP_OUTRO_LOCAL_AULA','IN_TRANSPORTE_PUBLICO','TP_RESPONSAVEL_TRANSPORTE',)

In [ ]:
# Define todas as colunas dos dados das turmas que não serão utilizadas neste projeto
cols_turma_drop = ('TX_HR_INICIAL','TX_MI_INICIAL','NO_TURMA','NU_DURACAO_TURMA','IN_REGULAR','IN_EJA','IN_PROFISSIONALIZANTE',                   'CO_CURSO_EDUC_PROFISSIONAL','IN_MAIS_EDUCACAO','IN_DIA_SEMANA_DOMINGO','IN_DIA_SEMANA_SEGUNDA',
                   'IN_DIA_SEMANA_TERCA','IN_DIA_SEMANA_QUARTA','IN_DIA_SEMANA_QUINTA','IN_DIA_SEMANA_SEXTA',                   'IN_DIA_SEMANA_SABADO','CO_TIPO_ATIVIDADE_1','CO_TIPO_ATIVIDADE_2','CO_TIPO_ATIVIDADE_3',
                   'CO_TIPO_ATIVIDADE_4','CO_TIPO_ATIVIDADE_5','CO_TIPO_ATIVIDADE_6','IN_BRAILLE','IN_RECURSOS_BAIXA_VISAO',                   'IN_ORIENTACAO_MOBILIDADE','IN_SINAIS','IN_COMUNICACAO_ALT_AUMENT','IN_ENRIQ_CURRICULAR','IN_SOROBAN',
                   'IN_INFORMATICA_ACESSIVEL','IN_PORT_ESCRITA','IN_AUTONOMIA_ESCOLAR','IN_DISC_QUIMICA','IN_DISC_FISICA',                   'IN_DISC_MATEMATICA','IN_DISC_BIOLOGIA','IN_DISC_CIENCIAS','IN_DISC_LINGUA_PORTUGUESA',
                   'IN_DISC_LINGUA_INGLES','IN_DISC_LINGUA_ESPANHOL','IN_DISC_LINGUA_FRANCES','IN_DISC_LINGUA_OUTRA',                   'IN_DISC_LINGUA_INDIGENA','IN_DISC_ARTES','IN_DISC_EDUCACAO_FISICA','IN_DISC_HISTORIA','IN_DISC_GEOGRAFIA',
                   'IN_DISC_FILOSOFIA','IN_DISC_ENSINO_RELIGIOSO','IN_DISC_ESTUDOS_SOCIAIS','IN_DISC_SOCIOLOGIA',                   'IN_DISC_EST_SOCIAIS_SOCIOLOGIA','IN_DISC_INFORMATICA_COMPUTACAO','IN_DISC_PROFISSIONALIZANTE',
                   'IN_DISC_ATENDIMENTO_ESPECIAIS','IN_DISC_DIVER_SOCIO_CULTURAL','IN_DISC_LIBRAS','IN_DISC_PEDAGOGICAS',                   'IN_DISC_OUTRAS','CO_REGIAO','CO_MESORREGIAO','CO_MICRORREGIAO','CO_UF','CO_MUNICIPIO','CO_DISTRITO',
                   'TP_DEPENDENCIA','TP_CATEGORIA_ESCOLA_PRIVADA','IN_CONVENIADA_PP','TP_CONVENIO_PODER_PUBLICO',                   'IN_MANT_ESCOLA_PRIVADA_EMP','IN_MANT_ESCOLA_PRIVADA_ONG','IN_MANT_ESCOLA_PRIVADA_SIND',
                   'IN_MANT_ESCOLA_PRIVADA_SIST_S','IN_MANT_ESCOLA_PRIVADA_S_FINS','TP_REGULAMENTACAO','TP_LOCALIZACAO_DIFERENCIADA','IN_EDUCACAO_INDIGENA','IN_DISC_PORT_SEGUNDA_LINGUA',
                   'IN_DISC_ESTAGIO_SUPERVISIONADO','IN_MANT_ESCOLA_PRIVADA_OSCIP','IN_MANT_ESCOLA_PRIV_ONG_OSCIP','NU_ANO_CENSO')

In [ ]:
# Define todas as colunas dos dados das escolas que não serão utilizadas neste projeto
cols_escola_drop = ('NU_ANO_CENSO','TP_FALTANTE','IN_MANT_ESCOLA_PRIVADA_EMP','IN_MANT_ESCOLA_PRIVADA_ONG','IN_MANT_ESCOLA_PRIVADA_SIND','IN_MANT_ESCOLA_PRIVADA_SIST_S','IN_MANT_ESCOLA_PRIVADA_S_FINS',
                    'CO_ESCOLA_SEDE_VINCULADA','CO_IES_OFERTANTE','TP_REGULAMENTACAO','IN_LOCAL_FUNC_PREDIO_ESCOLAR','TP_OCUPACAO_PREDIO_ESCOLAR','IN_REGULAR','IN_EJA','IN_PROFISSIONALIZANTE','IN_COMUM_CRECHE',
                    'IN_COMUM_PRE','IN_COMUM_FUND_AI','IN_COMUM_FUND_AF','IN_COMUM_MEDIO_MEDIO','IN_COMUM_MEDIO_INTEGRADO','IN_COMUM_MEDIO_NORMAL','IN_ESP_EXCLUSIVA_CRECHE','IN_ESP_EXCLUSIVA_PRE','IN_ESP_EXCLUSIVA_FUND_AI',
                    'IN_ESP_EXCLUSIVA_FUND_AF','IN_ESP_EXCLUSIVA_MEDIO_MEDIO','IN_ESP_EXCLUSIVA_MEDIO_INTEGR','IN_ESP_EXCLUSIVA_MEDIO_NORMAL','IN_COMUM_EJA_FUND','IN_COMUM_EJA_MEDIO','IN_COMUM_EJA_PROF',
                    'IN_ESP_EXCLUSIVA_EJA_FUND','IN_ESP_EXCLUSIVA_EJA_MEDIO','IN_ESP_EXCLUSIVA_EJA_PROF','IN_COMUM_PROF',                    'IN_ESP_EXCLUSIVA_PROF','IN_ORGAO_ASS_PAIS', 'IN_ORGAO_ASS_PAIS_MESTRES','IN_ORGAO_CONSELHO_ESCOLAR',
                    'IN_ORGAO_GREMIO_ESTUDANTIL','IN_ORGAO_OUTROS','IN_ORGAO_NENHUM','IN_RESERVA_PPI','IN_RESERVA_RENDA',                    'IN_RESERVA_PUBLICA','IN_RESERVA_PCD','IN_RESERVA_OUTROS','IN_RESERVA_NENHUMA','CO_LINGUA_INDIGENA_1',
                    'CO_LINGUA_INDIGENA_2','CO_LINGUA_INDIGENA_3','IN_MATERIAL_PED_MULTIMIDIA','IN_MATERIAL_PED_INFANTIL',                    'IN_MATERIAL_PED_CIENTIFICO','IN_MATERIAL_PED_DIFUSAO','IN_MATERIAL_PED_MUSICAL','IN_MATERIAL_PED_JOGOS',
                    'IN_MATERIAL_PED_ARTISTICAS','IN_MATERIAL_PED_DESPORTIVA','IN_MATERIAL_PED_INDIGENA','ID_MATRICULA',                    'IN_MATERIAL_PED_ETNICO','IN_MATERIAL_PED_CAMPO','IN_SERIE_ANO','IN_PERIODOS_SEMESTRAIS',
                    'IN_FUNDAMENTAL_CICLOS','IN_GRUPOS_NAO_SERIADOS','IN_MODULOS','IN_FORMACAO_ALTERNANCIA')

In [ ]:
# Define os parâmetros OPTIONS usados em todas as leituras de dados
config_file = {
    "header": True,
    "inferSchema": True,
    "delimiter": "|"
}

#### PRIMEIRA PARTE - DADOS DO CENSO 2019

In [ ]:
##### DADOS DOS DOCENTES - 2019

In [ ]:
# Lê os dados referentes aos docentes - centroeste
docentes_centroeste_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/DOCENTES_CO.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - nordeste
docentes_nordeste_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/DOCENTES_NORDESTE.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - norte
docentes_norte_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/DOCENTES_NORTE.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - sudeste
docentes_sudeste_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/DOCENTES_SUDESTE.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - sul
docentes_sul_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/DOCENTES_SUL.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas
docentes_centroeste_2019 = docentes_centroeste_2019.drop(*cols_docente_drop)
docentes_nordeste_2019 = docentes_nordeste_2019.drop(*cols_docente_drop)
docentes_norte_2019 = docentes_norte_2019.drop(*cols_docente_drop)
docentes_sudeste_2019 = docentes_sudeste_2019.drop(*cols_docente_drop)
docentes_sul_2019 = docentes_sul_2019.drop(*cols_docente_drop)

In [ ]:
# Faz a união de todos os dados coletados dos docentes
docentes_2019 = docentes_centroeste_2019.unionAll(docentes_nordeste_2019).unionAll(docentes_norte_2019).unionAll(docentes_sudeste_2019).unionAll(docentes_sul_2019)

In [ ]:
# Filtra os dados apenas com os Brasileiros Natos (definido no projeto)
docentes_2019 =  docentes_2019.filter('TP_NACIONALIDADE == "1"')

In [ ]:
# Dropa as tabelas referente a nacionalidade após a execução do filtro
docentes_2019 = docentes_2019.drop('TP_NACIONALIDADE', 'CO_PAIS_ORIGEM')

In [ ]:
# Salva os dados dos DOCENTES 2019 na pasta PRATA do bucket como arquivo PARQUET
docentes_2019.write.parquet('gs://igti-pa/prata/docentes_2019.parquet')

In [ ]:
##### DADOS DAS MATRÍCULAS - 2019

In [ ]:
# Lê os dados referentes as matrículas - centroeste
matriculas_centroeste_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/MATRICULA_CO.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - nordeste
matriculas_nordeste_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/MATRICULA_NORDESTE.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - norte
matriculas_norte_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/MATRICULA_NORTE.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - sudeste
matriculas_sudeste_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/MATRICULA_SUDESTE.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - sul
matriculas_sul_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/MATRICULA_SUL.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas de todos os dados coletados até então
matriculas_centroeste_2019 = matriculas_centroeste_2019.drop(*cols_matricula_drop)
matriculas_nordeste_2019 = matriculas_nordeste_2019.drop(*cols_matricula_drop)
matriculas_norte_2019 = matriculas_norte_2019.drop(*cols_matricula_drop)
matriculas_sudeste_2019 = matriculas_sudeste_2019.drop(*cols_matricula_drop)
matriculas_sul_2019 = matriculas_sul_2019.drop(*cols_matricula_drop)

In [ ]:
# Faz a união de todos os dados coletados das matrículas
matriculas_2019 = matriculas_centroeste_2019.unionAll(matriculas_nordeste_2019).unionAll(matriculas_norte_2019).unionAll(matriculas_sudeste_2019).unionAll(matriculas_sul_2019)

In [ ]:
# Filtra os dados apenas com os Brasileiros Natos (definido no projeto)
matriculas_2019 =  matriculas_2019.filter('TP_NACIONALIDADE == "1"')

In [ ]:
# Dropa as tabelas referente a nacionalidade após a execução do filtro
matriculas_2019 = matriculas_2019.drop('TP_NACIONALIDADE', 'CO_PAIS_ORIGEM')

In [ ]:
# Salva os dados das MATRÍCULAS 2019 na pasta PRATA do bucket como arquivo PARQUET
matriculas_2019.write.parquet('gs://igti-pa/prata/matriculas_2019.parquet')

In [ ]:
##### DADOS DAS TURMAS - 2019

In [ ]:
# Lê os dados referentes as turmas
turmas_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/TURMAS.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas
turmas_2019 = turmas_2019.drop(*cols_turma_drop)

In [ ]:
# Salva os dados das TURMAS 2019 na pasta PRATA do bucket como arquivo PARQUET
turmas_2019.write.parquet('gs://igti-pa/prata/turmas_2019.parquet')

In [ ]:
##### DADOS DAS ESCOLAS - 2019

In [ ]:
# Lê os dados referentes as escolas
escolas_2019 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2019/ESCOLAS.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas
escolas_2019 = escolas_2019.drop(*cols_escola_drop)

In [ ]:
# Salva os dados das ESCOLAS 2019 na pasta PRATA do bucket como arquivo PARQUET
escolas_2019.write.parquet('gs://igti-pa/prata/escolas_2019.parquet')

In [ ]:
##### TESTE DE INTEGRIDADE DOS DADOS - CENSO 2019

#### SEGUNDA PARTE - DADOS DO CENSO 2020

In [ ]:
##### DADOS DOS DOCENTES - 2020

In [ ]:
# Lê os dados referentes aos docentes - centroeste
docentes_centroeste_2020=(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/docentes_co.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - nordeste
docentes_nordeste_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/docentes_co.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - norte
docentes_norte_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/docentes_norte.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - sudeste
docentes_sudeste_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/docentes_sudeste.CSV")
)

In [ ]:
# Lê os dados referentes aos docentes - sul
docentes_sul_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/docentes_sul.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas
docentes_centroeste_2020 = docentes_centroeste_2020.drop(*cols_docente_drop)
docentes_nordeste_2020 = docentes_nordeste_2020.drop(*cols_docente_drop)
docentes_norte_2020 = docentes_norte_2020.drop(*cols_docente_drop)
docentes_sudeste_2020 = docentes_sudeste_2020.drop(*cols_docente_drop)
docentes_sul_2020 = docentes_sul_2020.drop(*cols_docente_drop)

In [ ]:
# Faz a união de todos os dados coletados dos docentes
docentes_2020 = docentes_centroeste_2020.unionAll(docentes_nordeste_2020).unionAll(docentes_norte_2020).unionAll(docentes_sudeste_2020).unionAll(docentes_sul_2020)

In [ ]:
# Filtra os dados apenas com os Brasileiros Natos (definido no projeto)
docentes_2020 =  docentes_2020.filter('TP_NACIONALIDADE == "1"')

In [ ]:
# Dropa as tabelas referente a nacionalidade após a execução do filtro
docentes_2020 = docentes_2020.drop('TP_NACIONALIDADE', 'CO_PAIS_ORIGEM')

In [ ]:
# Salva os dados dos DOCENTES 2020 na pasta PRATA do bucket como arquivo PARQUET
docentes_2020.write.parquet('gs://igti-pa/prata/docentes_2020.parquet')

In [ ]:
##### DADOS DAS MATRÍCULAS - 2020

In [ ]:
# Lê os dados referentes as matrículas - centroeste
matriculas_centroeste_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/matricula_co.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - nordeste
matriculas_nordeste_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/matricula_nordeste.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - norte
matriculas_norte_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/matricula_norte.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - sudeste
matriculas_sudeste_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/matricula_sudeste.CSV")
)

In [ ]:
# Lê os dados referentes as matrículas - sul
matriculas_sul_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/matricula_sul.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas de todos os dados coletados até então
matriculas_centroeste_2020 = matriculas_centroeste_2020.drop(*cols_matricula_drop)
matriculas_nordeste_2020 = matriculas_nordeste_2020.drop(*cols_matricula_drop)
matriculas_norte_2020 = matriculas_norte_2020.drop(*cols_matricula_drop)
matriculas_sudeste_2020 = matriculas_sudeste_2020.drop(*cols_matricula_drop)
matriculas_sul_2020 = matriculas_sul_2020.drop(*cols_matricula_drop)

In [ ]:
# Faz a união de todos os dados coletados das matrículas
matriculas_2020 = matriculas_centroeste_2020.unionAll(matriculas_nordeste_2020).unionAll(matriculas_norte_2020).unionAll(matriculas_sudeste_2020).unionAll(matriculas_sul_2020)

In [ ]:
# Filtra os dados apenas com os Brasileiros Natos (definido no projeto)
matriculas_2020 =  matriculas_2020.filter('TP_NACIONALIDADE == "1"')

In [ ]:
# Dropa as tabelas referente a nacionalidade após a execução do filtro
matriculas_2020 = matriculas_2020.drop('TP_NACIONALIDADE', 'CO_PAIS_ORIGEM')

In [ ]:
# Salva os dados das MATRÍCULAS 2020 na pasta PRATA do bucket como arquivo PARQUET
matriculas_2020.write.parquet('gs://igti-pa/prata/matriculas_2020.parquet')

In [ ]:
##### DADOS DAS TURMAS - 2020

In [ ]:
# Lê os dados referentes as turmas
turmas_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/turmas.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas
turmas_2020 = turmas_2020.drop(*cols_turma_drop)

In [ ]:
# Salva os dados das TURMAS 2020 na pasta PRATA do bucket como arquivo PARQUET
turmas_2020.write.parquet('gs://igti-pa/prata/turmas_2020.parquet')

In [ ]:
##### DADOS DAS ESCOLAS - 2020

In [ ]:
# Lê os dados referentes as escolas
escolas_2020 =(
    spark
    .read
    .format('csv')
    .options(**config_file)
    .load("gs://igti-pa/bronze/censo_2020/escolas.CSV")
)

In [ ]:
# Dropa todas as colunas que não serão utilizadas
escolas_2020 = escolas_2020.drop(*cols_escola_drop)

In [ ]:
# Salva os dados das ESCOLAS 2020 na pasta PRATA do bucket como arquivo PARQUET
escolas_2020.write.parquet('gs://igti-pa/prata/escolas_2020.parquet')

In [ ]:
##### TESTE DE INTEGRIDADE DOS DADOS - CENSO 2020

In [ ]:
spark.read.parquet('gs://igti-pa/prata/docentes_2020.parquet')

In [ ]:
spark.read.parquet('gs://igti-pa/prata/matriculas_2020.parquet')

In [ ]:
spark.read.parquet('gs://igti-pa/prata/turmas_2020.parquet')

In [ ]:
spark.read.parquet('gs://igti-pa/prata/escolas_2020.parquet')

#### TERCEIRA PARTE - TRANSFORMAÇÃO DOS DADOS CENSO 2019

In [58]:
##### DADOS DOS DOCENTES - 2019

In [1]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
docentes2019 = spark.read.parquet('gs://igti-pa/prata/docentes_2019.parquet')

In [2]:
docentes2019.count()

11515298

In [60]:
# Concatena as colunas NU_MES e NU_ANO para facilitar a visualização da DATA_NASCIMENTO
docentes2019 = docentes2019.withColumn('DATA_NASCIMENTO', f.concat_ws('-','NU_MES','NU_ANO').cast('string'))

In [61]:
# Dropa as colunas NU_MES e NU_ANO após a concatenação
docentes2019 = docentes2019.drop('NU_MES','NU_ANO')

In [62]:
# Redefine a ordem das colunas 
cols = ['DATA_NASCIMENTO','NU_IDADE_REFERENCIA','TP_SEXO','TP_COR_RACA','CO_UF_NASC','CO_MUNICIPIO_NASC','CO_UF_END','CO_MUNICIPIO_END','TP_ZONA_RESIDENCIAL','TP_LOCAL_RESID_DIFERENCIADA','IN_NECESSIDADE_ESPECIAL','TP_ESCOLARIDADE','TP_ENSINO_MEDIO','IN_COMPLEMENTACAO_PEDAGOGICA','IN_ESPECIALIZACAO', 'IN_MESTRADO','IN_DOUTORADO','IN_POS_NENHUM','IN_ESPECIFICO_GESTAO','ID_TURMA','TP_TIPO_DOCENTE','TP_TIPO_CONTRATACAO','TP_ETAPA_ENSINO','IN_ESPECIAL_EXCLUSIVA','CO_ENTIDADE']
docentes2019 = docentes2019[cols]

In [63]:
# Transforma a coluna DATA_NASCIMENTO para o tipo DATA
docentes2019 = docentes2019.withColumn('DATA_NASCIMENTO', f.to_date(docentes2019.DATA_NASCIMENTO, 'M-yyyy').alias('DATA_NASCIMENTO'))

In [64]:
# Altera o formato do tipo DATA da coluna DATA_NASCIMENTO 
docentes2019 = docentes2019.withColumn('DATA_NASCIMENTO', f.date_format('DATA_NASCIMENTO', 'dd-MM-yyyy').alias('DATA_NASCIMENTO'))

In [65]:
# Transforma todas as demais colunas para o tipo STRING
docentes2019 = docentes2019.withColumn("NU_IDADE_REFERENCIA", docentes2019["NU_IDADE_REFERENCIA"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_SEXO", docentes2019["TP_SEXO"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_COR_RACA", docentes2019["TP_COR_RACA"].cast('string'))
docentes2019 = docentes2019.withColumn("CO_UF_NASC", docentes2019["CO_UF_NASC"].cast('string'))
docentes2019 = docentes2019.withColumn("CO_MUNICIPIO_NASC", docentes2019["CO_MUNICIPIO_NASC"].cast('string'))
docentes2019 = docentes2019.withColumn("CO_UF_END", docentes2019["CO_UF_END"].cast('string'))
docentes2019 = docentes2019.withColumn("CO_MUNICIPIO_END", docentes2019["CO_MUNICIPIO_END"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_ZONA_RESIDENCIAL", docentes2019["TP_ZONA_RESIDENCIAL"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_LOCAL_RESID_DIFERENCIADA", docentes2019["TP_LOCAL_RESID_DIFERENCIADA"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_NECESSIDADE_ESPECIAL", docentes2019["IN_NECESSIDADE_ESPECIAL"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_ESCOLARIDADE", docentes2019["TP_ESCOLARIDADE"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_ENSINO_MEDIO", docentes2019["TP_ENSINO_MEDIO"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_COMPLEMENTACAO_PEDAGOGICA", docentes2019["IN_COMPLEMENTACAO_PEDAGOGICA"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_ESPECIALIZACAO", docentes2019["IN_ESPECIALIZACAO"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_MESTRADO", docentes2019["IN_MESTRADO"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_DOUTORADO", docentes2019["IN_DOUTORADO"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_POS_NENHUM", docentes2019["IN_POS_NENHUM"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_ESPECIFICO_GESTAO", docentes2019["IN_ESPECIFICO_GESTAO"].cast('string'))
docentes2019 = docentes2019.withColumn("ID_TURMA", docentes2019["ID_TURMA"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_TIPO_DOCENTE", docentes2019["TP_TIPO_DOCENTE"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_TIPO_CONTRATACAO", docentes2019["TP_TIPO_CONTRATACAO"].cast('string'))
docentes2019 = docentes2019.withColumn("TP_ETAPA_ENSINO", docentes2019["TP_ETAPA_ENSINO"].cast('string'))
docentes2019 = docentes2019.withColumn("IN_ESPECIAL_EXCLUSIVA", docentes2019["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
docentes2019 = docentes2019.withColumn("CO_ENTIDADE", docentes2019["CO_ENTIDADE"].cast('string'))

In [66]:
# Salva os dados dos DOCENTES 2019 na pasta OURO do bucket como arquivo PARQUET
docentes2019.write.parquet('gs://igti-pa/ouro/docentes2019.parquet')

In [72]:
##### DADOS DAS MATRÍCULAS - 2019

In [73]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
matriculas2019 = spark.read.parquet('gs://igti-pa/prata/matriculas_2019.parquet')

In [74]:
# Concatena as colunas NU_MES e NU_ANO para facilitar a visualização da DATA_NASCIMENTO
matriculas2019 = matriculas2019.withColumn('DATA_NASCIMENTO', f.concat_ws('-','NU_MES','NU_ANO').cast('string'))

In [75]:
# Dropa as colunas NU_MES e NU_ANO após a concatenação
matriculas2019 = matriculas2019.drop('NU_MES','NU_ANO')

In [76]:
# Redefine a ordem das colunas 
cols = ['ID_MATRICULA','DATA_NASCIMENTO','NU_IDADE_REFERENCIA','TP_SEXO','TP_COR_RACA','CO_UF_NASC','CO_MUNICIPIO_NASC','CO_UF_END','CO_MUNICIPIO_END','TP_ZONA_RESIDENCIAL','IN_NECESSIDADE_ESPECIAL','TP_ETAPA_ENSINO','IN_ESPECIAL_EXCLUSIVA','ID_TURMA','NU_DIAS_ATIVIDADE','TP_TIPO_ATENDIMENTO_TURMA','TP_TIPO_LOCAL_TURMA','CO_ENTIDADE']
matriculas2019 = matriculas2019[cols]

In [77]:
# Transforma a coluna DATA_NASCIMENTO para o tipo DATA
matriculas2019 = matriculas2019.withColumn('DATA_NASCIMENTO', f.to_date(matriculas2019.DATA_NASCIMENTO, 'M-yyyy').alias('DATA_NASCIMENTO'))

In [78]:
# Altera o formato do tipo DATA da coluna DATA_NASCIMENTO 
matriculas2019 = matriculas2019.withColumn('DATA_NASCIMENTO', f.date_format('DATA_NASCIMENTO', 'dd-MM-yyyy').alias('DATA_NASCIMENTO'))

In [79]:
# Transforma todas as demais colunas para o tipo STRING
matriculas2019 = matriculas2019.withColumn("ID_MATRICULA", matriculas2019["ID_MATRICULA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("NU_IDADE_REFERENCIA", matriculas2019["NU_IDADE_REFERENCIA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("TP_SEXO", matriculas2019["TP_SEXO"].cast('string'))
matriculas2019 = matriculas2019.withColumn("TP_COR_RACA", matriculas2019["TP_COR_RACA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("CO_UF_NASC", matriculas2019["CO_UF_NASC"].cast('string'))
matriculas2019 = matriculas2019.withColumn("CO_MUNICIPIO_NASC", matriculas2019["CO_MUNICIPIO_NASC"].cast('string'))
matriculas2019 = matriculas2019.withColumn("CO_UF_END", matriculas2019["CO_UF_END"].cast('string'))
matriculas2019 = matriculas2019.withColumn("CO_MUNICIPIO_END", matriculas2019["CO_MUNICIPIO_END"].cast('string'))
matriculas2019 = matriculas2019.withColumn("TP_ZONA_RESIDENCIAL", matriculas2019["TP_ZONA_RESIDENCIAL"].cast('string'))
matriculas2019 = matriculas2019.withColumn("IN_NECESSIDADE_ESPECIAL", matriculas2019["IN_NECESSIDADE_ESPECIAL"].cast('string'))
matriculas2019 = matriculas2019.withColumn("TP_ETAPA_ENSINO", matriculas2019["TP_ETAPA_ENSINO"].cast('string'))
matriculas2019 = matriculas2019.withColumn("IN_ESPECIAL_EXCLUSIVA", matriculas2019["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("ID_TURMA", matriculas2019["ID_TURMA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("NU_DIAS_ATIVIDADE", matriculas2019["NU_DIAS_ATIVIDADE"].cast('string'))
matriculas2019 = matriculas2019.withColumn("TP_TIPO_ATENDIMENTO_TURMA", matriculas2019["TP_TIPO_ATENDIMENTO_TURMA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("TP_TIPO_LOCAL_TURMA", matriculas2019["TP_TIPO_LOCAL_TURMA"].cast('string'))
matriculas2019 = matriculas2019.withColumn("CO_ENTIDADE", matriculas2019["CO_ENTIDADE"].cast('string'))

In [80]:
# Salva os dados das MATRICULAS 2019 na pasta OURO do bucket como arquivo PARQUET
matriculas2019.write.parquet('gs://igti-pa/ouro/matriculas2019.parquet')

In [81]:
##### DADOS DAS TURMAS - 2019

In [82]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
turmas2019 = spark.read.parquet('gs://igti-pa/prata/turmas_2019.parquet')

In [83]:
# Redefine a ordem das colunas 
cols = ['ID_TURMA','TP_LOCALIZACAO','TP_ETAPA_ENSINO','QT_MATRICULAS','NU_DIAS_ATIVIDADE','TP_MEDIACAO_DIDATICO_PEDAGO','TP_TIPO_ATENDIMENTO_TURMA','TP_TIPO_LOCAL_TURMA','IN_ESPECIAL_EXCLUSIVA','CO_ENTIDADE']
turmas2019 = turmas2019[cols]

In [84]:
# Transforma todas as demais colunas para o tipo STRING - MENOS A COLUNA QUANTIDADE (INT)
turmas2019 = turmas2019.withColumn("ID_TURMA", turmas2019["ID_TURMA"].cast('string'))
turmas2019 = turmas2019.withColumn("TP_MEDIACAO_DIDATICO_PEDAGO", turmas2019["TP_MEDIACAO_DIDATICO_PEDAGO"].cast('string'))
turmas2019 = turmas2019.withColumn("NU_DIAS_ATIVIDADE", turmas2019["NU_DIAS_ATIVIDADE"].cast('string'))
turmas2019 = turmas2019.withColumn("TP_TIPO_ATENDIMENTO_TURMA", turmas2019["TP_TIPO_ATENDIMENTO_TURMA"].cast('string'))
turmas2019 = turmas2019.withColumn("TP_TIPO_LOCAL_TURMA", turmas2019["TP_TIPO_LOCAL_TURMA"].cast('string'))
turmas2019 = turmas2019.withColumn("TP_ETAPA_ENSINO", turmas2019["TP_ETAPA_ENSINO"].cast('string'))
turmas2019 = turmas2019.withColumn("IN_ESPECIAL_EXCLUSIVA", turmas2019["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
turmas2019 = turmas2019.withColumn("QT_MATRICULAS", turmas2019["QT_MATRICULAS"].cast('int'))
turmas2019 = turmas2019.withColumn("CO_ENTIDADE", turmas2019["CO_ENTIDADE"].cast('string'))
turmas2019 = turmas2019.withColumn("TP_LOCALIZACAO", turmas2019["TP_LOCALIZACAO"].cast('string'))

In [85]:
# Salva os dados das TURMAS 2019 na pasta OURO do bucket como arquivo PARQUET
turmas2019.write.parquet('gs://igti-pa/ouro/turmas2019.parquet')

AnalysisException: path gs://igti-pa/ouro/turmas2019.parquet already exists.

In [86]:
##### DADOS DAS ESCOLAS - 2019

In [87]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
escolas2019 = spark.read.parquet('gs://igti-pa/prata/escolas_2019.parquet')

In [90]:
# Transforma as colunas de texto para STRING
escolas2019 = escolas2019.withColumn("CO_ENTIDADE", escolas2019["CO_ENTIDADE"].cast('string'))
escolas2019 = escolas2019.withColumn("NO_ENTIDADE", escolas2019["NO_ENTIDADE"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_ORGAO_REGIONAL", escolas2019["CO_ORGAO_REGIONAL"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_SITUACAO_FUNCIONAMENTO", escolas2019["TP_SITUACAO_FUNCIONAMENTO"].cast('string'))
escolas2019 = escolas2019.withColumn("DT_ANO_LETIVO_INICIO", escolas2019["DT_ANO_LETIVO_INICIO"].cast('string'))
escolas2019 = escolas2019.withColumn("DT_ANO_LETIVO_TERMINO", escolas2019["DT_ANO_LETIVO_TERMINO"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_REGIAO", escolas2019["CO_REGIAO"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_MESORREGIAO", escolas2019["CO_MESORREGIAO"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_MICRORREGIAO", escolas2019["CO_MICRORREGIAO"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_UF", escolas2019["CO_UF"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_MUNICIPIO", escolas2019["CO_MUNICIPIO"].cast('string'))
escolas2019 = escolas2019.withColumn("CO_DISTRITO", escolas2019["CO_DISTRITO"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_DEPENDENCIA", escolas2019["TP_DEPENDENCIA"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_LOCALIZACAO", escolas2019["TP_LOCALIZACAO"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_LOCALIZACAO_DIFERENCIADA", escolas2019["TP_LOCALIZACAO_DIFERENCIADA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_VINCULO_SECRETARIA_EDUCACAO", escolas2019["IN_VINCULO_SECRETARIA_EDUCACAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_VINCULO_SEGURANCA_PUBLICA", escolas2019["IN_VINCULO_SEGURANCA_PUBLICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_VINCULO_SECRETARIA_SAUDE", escolas2019["IN_VINCULO_SECRETARIA_SAUDE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_VINCULO_OUTRO_ORGAO", escolas2019["IN_VINCULO_OUTRO_ORGAO"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_CATEGORIA_ESCOLA_PRIVADA", escolas2019["TP_CATEGORIA_ESCOLA_PRIVADA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_CONVENIADA_PP", escolas2019["IN_CONVENIADA_PP"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_CONVENIO_PODER_PUBLICO", escolas2019["TP_CONVENIO_PODER_PUBLICO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_MANT_ESCOLA_PRIVADA_OSCIP", escolas2019["IN_MANT_ESCOLA_PRIVADA_OSCIP"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_MANT_ESCOLA_PRIV_ONG_OSCIP", escolas2019["IN_MANT_ESCOLA_PRIV_ONG_OSCIP"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_RESPONSAVEL_REGULAMENTACAO", escolas2019["TP_RESPONSAVEL_REGULAMENTACAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LOCAL_FUNC_SOCIOEDUCATIVO", escolas2019["IN_LOCAL_FUNC_SOCIOEDUCATIVO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LOCAL_FUNC_UNID_PRISIONAL", escolas2019["IN_LOCAL_FUNC_UNID_PRISIONAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LOCAL_FUNC_PRISIONAL_SOCIO", escolas2019["IN_LOCAL_FUNC_PRISIONAL_SOCIO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LOCAL_FUNC_GALPAO", escolas2019["IN_LOCAL_FUNC_GALPAO"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_OCUPACAO_GALPAO", escolas2019["TP_OCUPACAO_GALPAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LOCAL_FUNC_SALAS_OUTRA_ESC", escolas2019["IN_LOCAL_FUNC_SALAS_OUTRA_ESC"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LOCAL_FUNC_OUTROS", escolas2019["IN_LOCAL_FUNC_OUTROS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_PREDIO_COMPARTILHADO", escolas2019["IN_PREDIO_COMPARTILHADO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AGUA_POTAVEL", escolas2019["IN_AGUA_POTAVEL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AGUA_REDE_PUBLICA", escolas2019["IN_AGUA_REDE_PUBLICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AGUA_POCO_ARTESIANO", escolas2019["IN_AGUA_POCO_ARTESIANO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AGUA_CACIMBA", escolas2019["IN_AGUA_CACIMBA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AGUA_FONTE_RIO", escolas2019["IN_AGUA_FONTE_RIO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AGUA_INEXISTENTE", escolas2019["IN_AGUA_INEXISTENTE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ENERGIA_REDE_PUBLICA", escolas2019["IN_ENERGIA_REDE_PUBLICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ENERGIA_GERADOR_FOSSIL", escolas2019["IN_ENERGIA_GERADOR_FOSSIL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ENERGIA_RENOVAVEL", escolas2019["IN_ENERGIA_RENOVAVEL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ENERGIA_INEXISTENTE", escolas2019["IN_ENERGIA_INEXISTENTE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESGOTO_REDE_PUBLICA", escolas2019["IN_ESGOTO_REDE_PUBLICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESGOTO_FOSSA_SEPTICA", escolas2019["IN_ESGOTO_FOSSA_SEPTICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESGOTO_FOSSA_COMUM", escolas2019["IN_ESGOTO_FOSSA_COMUM"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESGOTO_FOSSA", escolas2019["IN_ESGOTO_FOSSA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESGOTO_INEXISTENTE", escolas2019["IN_ESGOTO_INEXISTENTE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LIXO_SERVICO_COLETA", escolas2019["IN_LIXO_SERVICO_COLETA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LIXO_QUEIMA", escolas2019["IN_LIXO_QUEIMA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LIXO_ENTERRA", escolas2019["IN_LIXO_ENTERRA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LIXO_DESTINO_FINAL_PUBLICO", escolas2019["IN_LIXO_DESTINO_FINAL_PUBLICO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LIXO_DESCARTA_OUTRA_AREA", escolas2019["IN_LIXO_DESCARTA_OUTRA_AREA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_TRATAMENTO_LIXO_SEPARACAO", escolas2019["IN_TRATAMENTO_LIXO_SEPARACAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_TRATAMENTO_LIXO_REUTILIZA", escolas2019["IN_TRATAMENTO_LIXO_REUTILIZA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_TRATAMENTO_LIXO_RECICLAGEM", escolas2019["IN_TRATAMENTO_LIXO_RECICLAGEM"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_TRATAMENTO_LIXO_INEXISTENTE", escolas2019["IN_TRATAMENTO_LIXO_INEXISTENTE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ALMOXARIFADO", escolas2019["IN_ALMOXARIFADO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AREA_VERDE", escolas2019["IN_AREA_VERDE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_AUDITORIO", escolas2019["IN_AUDITORIO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BANHEIRO", escolas2019["IN_BANHEIRO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BANHEIRO_EI", escolas2019["IN_BANHEIRO_EI"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BANHEIRO_PNE", escolas2019["IN_BANHEIRO_PNE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BANHEIRO_FUNCIONARIOS", escolas2019["IN_BANHEIRO_FUNCIONARIOS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BANHEIRO_CHUVEIRO", escolas2019["IN_BANHEIRO_CHUVEIRO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BIBLIOTECA", escolas2019["IN_BIBLIOTECA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BIBLIOTECA_SALA_LEITURA", escolas2019["IN_BIBLIOTECA_SALA_LEITURA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_COZINHA", escolas2019["IN_COZINHA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_DESPENSA", escolas2019["IN_DESPENSA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_DORMITORIO_ALUNO", escolas2019["IN_DORMITORIO_ALUNO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_DORMITORIO_PROFESSOR", escolas2019["IN_DORMITORIO_PROFESSOR"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LABORATORIO_CIENCIAS", escolas2019["IN_LABORATORIO_CIENCIAS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_LABORATORIO_INFORMATICA", escolas2019["IN_LABORATORIO_INFORMATICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_PATIO_COBERTO", escolas2019["IN_PATIO_COBERTO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_PATIO_DESCOBERTO", escolas2019["IN_PATIO_DESCOBERTO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_PARQUE_INFANTIL", escolas2019["IN_PARQUE_INFANTIL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_PISCINA", escolas2019["IN_PISCINA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_QUADRA_ESPORTES", escolas2019["IN_QUADRA_ESPORTES"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_QUADRA_ESPORTES_COBERTA", escolas2019["IN_QUADRA_ESPORTES_COBERTA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_QUADRA_ESPORTES_DESCOBERTA", escolas2019["IN_QUADRA_ESPORTES_DESCOBERTA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_REFEITORIO", escolas2019["IN_REFEITORIO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_ATELIE_ARTES", escolas2019["IN_SALA_ATELIE_ARTES"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_MUSICA_CORAL", escolas2019["IN_SALA_MUSICA_CORAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_ESTUDIO_DANCA", escolas2019["IN_SALA_ESTUDIO_DANCA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_MULTIUSO", escolas2019["IN_SALA_MULTIUSO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_DIRETORIA", escolas2019["IN_SALA_DIRETORIA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_LEITURA", escolas2019["IN_SALA_LEITURA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_PROFESSOR", escolas2019["IN_SALA_PROFESSOR"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_REPOUSO_ALUNO", escolas2019["IN_SALA_REPOUSO_ALUNO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SECRETARIA", escolas2019["IN_SECRETARIA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_SALA_ATENDIMENTO_ESPECIAL", escolas2019["IN_SALA_ATENDIMENTO_ESPECIAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_TERREIRAO", escolas2019["IN_TERREIRAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_VIVEIRO", escolas2019["IN_VIVEIRO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_DEPENDENCIAS_OUTRAS", escolas2019["IN_DEPENDENCIAS_OUTRAS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_CORRIMAO", escolas2019["IN_ACESSIBILIDADE_CORRIMAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_ELEVADOR", escolas2019["IN_ACESSIBILIDADE_ELEVADOR"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_PISOS_TATEIS", escolas2019["IN_ACESSIBILIDADE_PISOS_TATEIS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_VAO_LIVRE", escolas2019["IN_ACESSIBILIDADE_VAO_LIVRE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_RAMPAS", escolas2019["IN_ACESSIBILIDADE_RAMPAS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_SINAL_SONORO", escolas2019["IN_ACESSIBILIDADE_SINAL_SONORO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_SINAL_TATIL", escolas2019["IN_ACESSIBILIDADE_SINAL_TATIL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_SINAL_VISUAL", escolas2019["IN_ACESSIBILIDADE_SINAL_VISUAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSIBILIDADE_INEXISTENTE", escolas2019["IN_ACESSIBILIDADE_INEXISTENTE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_COMPUTADOR", escolas2019["IN_COMPUTADOR"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_COPIADORA", escolas2019["IN_EQUIP_COPIADORA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_IMPRESSORA", escolas2019["IN_EQUIP_IMPRESSORA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_IMPRESSORA_MULT", escolas2019["IN_EQUIP_IMPRESSORA_MULT"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_SCANNER", escolas2019["IN_EQUIP_DVD"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_SOM", escolas2019["IN_EQUIP_TV"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_LOUSA_DIGITAL", escolas2019["IN_EQUIP_LOUSA_DIGITAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_MULTIMIDIA", escolas2019["IN_EQUIP_MULTIMIDIA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_DESKTOP_ALUNO", escolas2019["IN_DESKTOP_ALUNO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_COMP_PORTATIL_ALUNO", escolas2019["IN_COMP_PORTATIL_ALUNO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_TABLET_ALUNO", escolas2019["IN_TABLET_ALUNO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_INTERNET", escolas2019["IN_INTERNET"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_INTERNET_ALUNOS", escolas2019["IN_INTERNET_ALUNOS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_INTERNET_ADMINISTRATIVO", escolas2019["IN_INTERNET_ADMINISTRATIVO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_INTERNET_APRENDIZAGEM", escolas2019["IN_INTERNET_APRENDIZAGEM"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_INTERNET_COMUNIDADE", escolas2019["IN_INTERNET_COMUNIDADE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACESSO_INTERNET_COMPUTADOR", escolas2019["IN_ACESSO_INTERNET_COMPUTADOR"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ACES_INTERNET_DISP_PESSOAIS", escolas2019["IN_ACES_INTERNET_DISP_PESSOAIS"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_REDE_LOCAL", escolas2019["TP_REDE_LOCAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_BANDA_LARGA", escolas2019["IN_BANDA_LARGA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ALIMENTACAO", escolas2019["IN_ALIMENTACAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EDUCACAO_INDIGENA", escolas2019["IN_EDUCACAO_INDIGENA"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_INDIGENA_LINGUA", escolas2019["TP_INDIGENA_LINGUA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EXAME_SELECAO", escolas2019["IN_EXAME_SELECAO"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_REDES_SOCIAIS", escolas2019["IN_REDES_SOCIAIS"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESPACO_ATIVIDADE", escolas2019["IN_ESPACO_ATIVIDADE"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESPACO_EQUIPAMENTO", escolas2019["IN_ESPACO_EQUIPAMENTO"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_PROPOSTA_PEDAGOGICA", escolas2019["TP_PROPOSTA_PEDAGOGICA"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_AEE", escolas2019["TP_AEE"].cast('string'))
escolas2019 = escolas2019.withColumn("TP_ATIVIDADE_COMPLEMENTAR", escolas2019["TP_ATIVIDADE_COMPLEMENTAR"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_MEDIACAO_PRESENCIAL", escolas2019["IN_MEDIACAO_PRESENCIAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_MEDIACAO_SEMIPRESENCIAL", escolas2019["IN_MEDIACAO_SEMIPRESENCIAL"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_MEDIACAO_EAD", escolas2019["IN_MEDIACAO_EAD"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_ESPECIAL_EXCLUSIVA", escolas2019["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_PARABOLICA", escolas2019["IN_EQUIP_PARABOLICA"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_DVD", escolas2019["IN_EQUIP_DVD"].cast('string'))
escolas2019 = escolas2019.withColumn("IN_EQUIP_TV", escolas2019["IN_EQUIP_TV"].cast('string'))

In [91]:
# Transforma as colunas de quantidade para INT
escolas2019 = escolas2019.withColumn("QT_SALAS_UTILIZADAS_DENTRO", escolas2019["QT_SALAS_UTILIZADAS_DENTRO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_SALAS_UTILIZADAS_FORA", escolas2019["QT_SALAS_UTILIZADAS_FORA"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_SALAS_UTILIZADAS", escolas2019["QT_SALAS_UTILIZADAS"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_SALAS_UTILIZA_CLIMATIZADAS", escolas2019["QT_SALAS_UTILIZA_CLIMATIZADAS"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_SALAS_UTILIZADAS_ACESSIVEIS", escolas2019["QT_SALAS_UTILIZADAS_ACESSIVEIS"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_EQUIP_DVD", escolas2019["QT_EQUIP_DVD"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_EQUIP_SOM", escolas2019["QT_EQUIP_SOM"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_EQUIP_TV", escolas2019["QT_EQUIP_TV"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_EQUIP_MULTIMIDIA", escolas2019["QT_EQUIP_MULTIMIDIA"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_EQUIP_LOUSA_DIGITAL", escolas2019["QT_EQUIP_LOUSA_DIGITAL"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_DESKTOP_ALUNO", escolas2019["QT_DESKTOP_ALUNO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_COMP_PORTATIL_ALUNO", escolas2019["QT_COMP_PORTATIL_ALUNO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_TABLET_ALUNO", escolas2019["QT_TABLET_ALUNO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_ADMINISTRATIVOS", escolas2019["QT_PROF_ADMINISTRATIVOS"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_SERVICOS_GERAIS", escolas2019["QT_PROF_SERVICOS_GERAIS"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_BIBLIOTECARIO", escolas2019["QT_PROF_BIBLIOTECARIO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_SAUDE", escolas2019["QT_PROF_SAUDE"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_COORDENADOR", escolas2019["QT_PROF_COORDENADOR"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_FONAUDIOLOGO", escolas2019["QT_PROF_FONAUDIOLOGO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_NUTRICIONISTA", escolas2019["QT_PROF_NUTRICIONISTA"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_PSICOLOGO", escolas2019["QT_PROF_PSICOLOGO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_ALIMENTACAO", escolas2019["QT_PROF_ALIMENTACAO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_PEDAGOGIA", escolas2019["QT_PROF_PEDAGOGIA"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_SECRETARIO", escolas2019["QT_PROF_SECRETARIO"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_SEGURANCA", escolas2019["QT_PROF_SEGURANCA"].cast('int'))
escolas2019 = escolas2019.withColumn("QT_PROF_MONITORES", escolas2019["QT_PROF_MONITORES"].cast('int'))

In [92]:
# Salva os dados das ESCOLAS 2019 na pasta OURO do bucket como arquivo PARQUET
escolas2019.write.parquet('gs://igti-pa/ouro/escolas2019.parquet')

In [93]:
##### TESTE DE INTEGRIDADE DOS DADOS - CENSO 2019

In [94]:
spark.read.parquet('gs://igti-pa/ouro/docentes2019.parquet')

DataFrame[DATA_NASCIMENTO: string, NU_IDADE_REFERENCIA: string, TP_SEXO: string, TP_COR_RACA: string, CO_UF_NASC: string, CO_MUNICIPIO_NASC: string, CO_UF_END: string, CO_MUNICIPIO_END: string, TP_ZONA_RESIDENCIAL: string, TP_LOCAL_RESID_DIFERENCIADA: string, IN_NECESSIDADE_ESPECIAL: string, TP_ESCOLARIDADE: string, TP_ENSINO_MEDIO: string, IN_COMPLEMENTACAO_PEDAGOGICA: string, IN_ESPECIALIZACAO: string, IN_MESTRADO: string, IN_DOUTORADO: string, IN_POS_NENHUM: string, IN_ESPECIFICO_GESTAO: string, ID_TURMA: string, TP_TIPO_DOCENTE: string, TP_TIPO_CONTRATACAO: string, TP_ETAPA_ENSINO: string, IN_ESPECIAL_EXCLUSIVA: string, CO_ENTIDADE: string]

In [95]:
spark.read.parquet('gs://igti-pa/ouro/matriculas2019.parquet')

DataFrame[ID_MATRICULA: string, DATA_NASCIMENTO: string, NU_IDADE_REFERENCIA: string, TP_SEXO: string, TP_COR_RACA: string, CO_UF_NASC: string, CO_MUNICIPIO_NASC: string, CO_UF_END: string, CO_MUNICIPIO_END: string, TP_ZONA_RESIDENCIAL: string, IN_NECESSIDADE_ESPECIAL: string, TP_ETAPA_ENSINO: string, IN_ESPECIAL_EXCLUSIVA: string, ID_TURMA: string, NU_DIAS_ATIVIDADE: string, TP_TIPO_ATENDIMENTO_TURMA: string, TP_TIPO_LOCAL_TURMA: string, CO_ENTIDADE: string]

In [96]:
spark.read.parquet('gs://igti-pa/ouro/turmas2019.parquet')

DataFrame[ID_TURMA: string, TP_LOCALIZACAO: string, TP_ETAPA_ENSINO: string, QT_MATRICULAS: int, NU_DIAS_ATIVIDADE: string, TP_MEDIACAO_DIDATICO_PEDAGO: string, TP_TIPO_ATENDIMENTO_TURMA: string, TP_TIPO_LOCAL_TURMA: string, IN_ESPECIAL_EXCLUSIVA: string, CO_ENTIDADE: string]

In [97]:
spark.read.parquet('gs://igti-pa/ouro/escolas2019.parquet')

DataFrame[CO_ENTIDADE: string, NO_ENTIDADE: string, CO_ORGAO_REGIONAL: string, TP_SITUACAO_FUNCIONAMENTO: string, DT_ANO_LETIVO_INICIO: string, DT_ANO_LETIVO_TERMINO: string, CO_REGIAO: string, CO_MESORREGIAO: string, CO_MICRORREGIAO: string, CO_UF: string, CO_MUNICIPIO: string, CO_DISTRITO: string, TP_DEPENDENCIA: string, TP_LOCALIZACAO: string, TP_LOCALIZACAO_DIFERENCIADA: string, IN_VINCULO_SECRETARIA_EDUCACAO: string, IN_VINCULO_SEGURANCA_PUBLICA: string, IN_VINCULO_SECRETARIA_SAUDE: string, IN_VINCULO_OUTRO_ORGAO: string, TP_CATEGORIA_ESCOLA_PRIVADA: string, IN_CONVENIADA_PP: string, TP_CONVENIO_PODER_PUBLICO: string, IN_MANT_ESCOLA_PRIVADA_OSCIP: string, IN_MANT_ESCOLA_PRIV_ONG_OSCIP: string, TP_RESPONSAVEL_REGULAMENTACAO: string, IN_LOCAL_FUNC_SOCIOEDUCATIVO: string, IN_LOCAL_FUNC_UNID_PRISIONAL: string, IN_LOCAL_FUNC_PRISIONAL_SOCIO: string, IN_LOCAL_FUNC_GALPAO: string, TP_OCUPACAO_GALPAO: string, IN_LOCAL_FUNC_SALAS_OUTRA_ESC: string, IN_LOCAL_FUNC_OUTROS: string, IN_PREDIO_C

#### QUARTA PARTE - TRANSFORMAÇÃO DOS DADOS CENSO 2020

In [98]:
##### DADOS DOS DOCENTES - 2020

In [99]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
docentes2020 = spark.read.parquet('gs://igti-pa/prata/docentes_2020.parquet')

In [100]:
# Concatena as colunas NU_MES e NU_ANO para facilitar a visualização da DATA_NASCIMENTO
docentes2020 = docentes2020.withColumn('DATA_NASCIMENTO', f.concat_ws('-','NU_MES','NU_ANO').cast('string'))

In [101]:
# Dropa as colunas NU_MES e NU_ANO após a concatenação
docentes2020 = docentes2020.drop('NU_MES','NU_ANO')

In [102]:
# Redefine a ordem das colunas 
cols = ['DATA_NASCIMENTO','NU_IDADE_REFERENCIA','TP_SEXO','TP_COR_RACA','CO_UF_NASC','CO_MUNICIPIO_NASC','CO_UF_END','CO_MUNICIPIO_END','TP_ZONA_RESIDENCIAL','IN_NECESSIDADE_ESPECIAL','TP_ESCOLARIDADE','TP_ENSINO_MEDIO','IN_ESPECIALIZACAO', 'IN_MESTRADO','IN_DOUTORADO','IN_POS_NENHUM','IN_ESPECIFICO_GESTAO','ID_TURMA','TP_TIPO_DOCENTE','TP_TIPO_CONTRATACAO','TP_ETAPA_ENSINO','IN_ESPECIAL_EXCLUSIVA','CO_ENTIDADE']
docentes2020 = docentes2020[cols]

In [103]:
# Transforma a coluna DATA_NASCIMENTO para o tipo DATA
docentes2020 = docentes2020.withColumn('DATA_NASCIMENTO', f.to_date(docentes2020.DATA_NASCIMENTO, 'M-yyyy').alias('DATA_NASCIMENTO'))

In [104]:
# Altera o formato do tipo DATA da coluna DATA_NASCIMENTO 
docentes2020 = docentes2020.withColumn('DATA_NASCIMENTO', f.date_format('DATA_NASCIMENTO', 'dd-MM-yyyy').alias('DATA_NASCIMENTO'))

In [105]:
# Transforma todas as demais colunas para o tipo STRING
docentes2020 = docentes2020.withColumn("NU_IDADE_REFERENCIA", docentes2020["NU_IDADE_REFERENCIA"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_SEXO", docentes2020["TP_SEXO"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_COR_RACA", docentes2020["TP_COR_RACA"].cast('string'))
docentes2020 = docentes2020.withColumn("CO_UF_NASC", docentes2020["CO_UF_NASC"].cast('string'))
docentes2020 = docentes2020.withColumn("CO_MUNICIPIO_NASC", docentes2020["CO_MUNICIPIO_NASC"].cast('string'))
docentes2020 = docentes2020.withColumn("CO_UF_END", docentes2020["CO_UF_END"].cast('string'))
docentes2020 = docentes2020.withColumn("CO_MUNICIPIO_END", docentes2020["CO_MUNICIPIO_END"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_ZONA_RESIDENCIAL", docentes2020["TP_ZONA_RESIDENCIAL"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_NECESSIDADE_ESPECIAL", docentes2020["IN_NECESSIDADE_ESPECIAL"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_ESCOLARIDADE", docentes2020["TP_ESCOLARIDADE"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_ENSINO_MEDIO", docentes2020["TP_ENSINO_MEDIO"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_ESPECIALIZACAO", docentes2020["IN_ESPECIALIZACAO"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_MESTRADO", docentes2020["IN_MESTRADO"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_DOUTORADO", docentes2020["IN_DOUTORADO"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_POS_NENHUM", docentes2020["IN_POS_NENHUM"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_ESPECIFICO_GESTAO", docentes2020["IN_ESPECIFICO_GESTAO"].cast('string'))
docentes2020 = docentes2020.withColumn("ID_TURMA", docentes2020["ID_TURMA"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_TIPO_DOCENTE", docentes2020["TP_TIPO_DOCENTE"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_TIPO_CONTRATACAO", docentes2020["TP_TIPO_CONTRATACAO"].cast('string'))
docentes2020 = docentes2020.withColumn("TP_ETAPA_ENSINO", docentes2020["TP_ETAPA_ENSINO"].cast('string'))
docentes2020 = docentes2020.withColumn("IN_ESPECIAL_EXCLUSIVA", docentes2020["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
docentes2020 = docentes2020.withColumn("CO_ENTIDADE", docentes2020["CO_ENTIDADE"].cast('string'))

In [106]:
# Salva os dados dos DOCENTES 2019 na pasta OURO do bucket como arquivo PARQUET
docentes2020.write.parquet('gs://igti-pa/ouro/docentes2020.parquet')

In [100]:
##### DADOS DAS MATRÍCULAS - 2020

In [101]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
matriculas2020 = spark.read.parquet('gs://igti-pa/prata/matriculas_2020.parquet')

In [102]:
# Concatena as colunas NU_MES e NU_ANO para facilitar a visualização da DATA_NASCIMENTO
matriculas2020 = matriculas2020.withColumn('DATA_NASCIMENTO', f.concat_ws('-','NU_MES','NU_ANO').cast('string'))

In [103]:
# Dropa as colunas NU_MES e NU_ANO após a concatenação
matriculas2020 = matriculas2020.drop('NU_MES','NU_ANO')

In [104]:
# Redefine a ordem das colunas 
cols = ['ID_MATRICULA','DATA_NASCIMENTO','NU_IDADE_REFERENCIA','TP_SEXO','TP_COR_RACA','CO_UF_NASC','CO_MUNICIPIO_NASC','CO_UF_END','CO_MUNICIPIO_END','TP_ZONA_RESIDENCIAL','IN_NECESSIDADE_ESPECIAL','TP_ETAPA_ENSINO','IN_ESPECIAL_EXCLUSIVA','ID_TURMA','NU_DIAS_ATIVIDADE','TP_TIPO_ATENDIMENTO_TURMA','TP_TIPO_LOCAL_TURMA','CO_ENTIDADE']
matriculas2020 = matriculas2020[cols]

In [105]:
# Transforma a coluna DATA_NASCIMENTO para o tipo DATA
matriculas2020 = matriculas2020.withColumn('DATA_NASCIMENTO', f.to_date(matriculas2020.DATA_NASCIMENTO, 'M-yyyy').alias('DATA_NASCIMENTO'))

In [106]:
# Altera o formato do tipo DATA da coluna DATA_NASCIMENTO 
matriculas2020 = matriculas2020.withColumn('DATA_NASCIMENTO', f.date_format('DATA_NASCIMENTO', 'dd-MM-yyyy').alias('DATA_NASCIMENTO'))

In [107]:
# Transforma todas as demais colunas para o tipo STRING
matriculas2020 = matriculas2020.withColumn("ID_MATRICULA", matriculas2020["ID_MATRICULA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("NU_IDADE_REFERENCIA", matriculas2020["NU_IDADE_REFERENCIA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("TP_SEXO", matriculas2020["TP_SEXO"].cast('string'))
matriculas2020 = matriculas2020.withColumn("TP_COR_RACA", matriculas2020["TP_COR_RACA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("CO_UF_NASC", matriculas2020["CO_UF_NASC"].cast('string'))
matriculas2020 = matriculas2020.withColumn("CO_MUNICIPIO_NASC", matriculas2020["CO_MUNICIPIO_NASC"].cast('string'))
matriculas2020 = matriculas2020.withColumn("CO_UF_END", matriculas2020["CO_UF_END"].cast('string'))
matriculas2020 = matriculas2020.withColumn("CO_MUNICIPIO_END", matriculas2020["CO_MUNICIPIO_END"].cast('string'))
matriculas2020 = matriculas2020.withColumn("TP_ZONA_RESIDENCIAL", matriculas2020["TP_ZONA_RESIDENCIAL"].cast('string'))
matriculas2020 = matriculas2020.withColumn("IN_NECESSIDADE_ESPECIAL", matriculas2020["IN_NECESSIDADE_ESPECIAL"].cast('string'))
matriculas2020 = matriculas2020.withColumn("TP_ETAPA_ENSINO", matriculas2020["TP_ETAPA_ENSINO"].cast('string'))
matriculas2020 = matriculas2020.withColumn("IN_ESPECIAL_EXCLUSIVA", matriculas2020["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("ID_TURMA", matriculas2020["ID_TURMA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("NU_DIAS_ATIVIDADE", matriculas2020["NU_DIAS_ATIVIDADE"].cast('string'))
matriculas2020 = matriculas2020.withColumn("TP_TIPO_ATENDIMENTO_TURMA", matriculas2020["TP_TIPO_ATENDIMENTO_TURMA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("TP_TIPO_LOCAL_TURMA", matriculas2020["TP_TIPO_LOCAL_TURMA"].cast('string'))
matriculas2020 = matriculas2020.withColumn("CO_ENTIDADE", matriculas2020["CO_ENTIDADE"].cast('string'))

In [108]:
# Salva os dados das MATRICULAS 2019 na pasta OURO do bucket como arquivo PARQUET
matriculas2020.write.parquet('gs://igti-pa/ouro/matriculas2020.parquet')

In [109]:
##### DADOS DAS TURMAS - 2020

In [110]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
turmas2020 = spark.read.parquet('gs://igti-pa/prata/turmas_2020.parquet')

In [111]:
# Redefine a ordem das colunas 
cols = ['ID_TURMA','TP_LOCALIZACAO','TP_ETAPA_ENSINO','QT_MATRICULAS','NU_DIAS_ATIVIDADE','TP_MEDIACAO_DIDATICO_PEDAGO','TP_TIPO_ATENDIMENTO_TURMA','TP_TIPO_LOCAL_TURMA','IN_ESPECIAL_EXCLUSIVA','CO_ENTIDADE']
turmas2020 = turmas2020[cols]

In [112]:
# Transforma todas as demais colunas para o tipo STRING - MENOS A COLUNA QUANTIDADE (INT)
turmas2020 = turmas2020.withColumn("ID_TURMA", turmas2020["ID_TURMA"].cast('string'))
turmas2020 = turmas2020.withColumn("TP_MEDIACAO_DIDATICO_PEDAGO", turmas2020["TP_MEDIACAO_DIDATICO_PEDAGO"].cast('string'))
turmas2020 = turmas2020.withColumn("NU_DIAS_ATIVIDADE", turmas2020["NU_DIAS_ATIVIDADE"].cast('string'))
turmas2020 = turmas2020.withColumn("TP_TIPO_ATENDIMENTO_TURMA", turmas2020["TP_TIPO_ATENDIMENTO_TURMA"].cast('string'))
turmas2020 = turmas2020.withColumn("TP_TIPO_LOCAL_TURMA", turmas2020["TP_TIPO_LOCAL_TURMA"].cast('string'))
turmas2020 = turmas2020.withColumn("TP_ETAPA_ENSINO", turmas2020["TP_ETAPA_ENSINO"].cast('string'))
turmas2020 = turmas2020.withColumn("IN_ESPECIAL_EXCLUSIVA", turmas2020["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
turmas2020 = turmas2020.withColumn("QT_MATRICULAS", turmas2020["QT_MATRICULAS"].cast('int'))
turmas2020 = turmas2020.withColumn("CO_ENTIDADE", turmas2020["CO_ENTIDADE"].cast('string'))
turmas2020 = turmas2020.withColumn("TP_LOCALIZACAO", turmas2020["TP_LOCALIZACAO"].cast('string'))

In [113]:
# Salva os dados das TURMAS 2019 na pasta OURO do bucket como arquivo PARQUET
turmas2020.write.parquet('gs://igti-pa/ouro/turmas2020.parquet')

In [114]:
##### DADOS DAS ESCOLAS - 2020

In [115]:
# Faz a leitura dos dados na pasta PRATA do bucket GCP
escolas2020 = spark.read.parquet('gs://igti-pa/prata/escolas_2020.parquet')

In [116]:
# Transforma as colunas de texto para STRING
escolas2020 = escolas2020.withColumn("CO_ENTIDADE", escolas2020["CO_ENTIDADE"].cast('string'))
escolas2020 = escolas2020.withColumn("NO_ENTIDADE", escolas2020["NO_ENTIDADE"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_ORGAO_REGIONAL", escolas2020["CO_ORGAO_REGIONAL"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_SITUACAO_FUNCIONAMENTO", escolas2020["TP_SITUACAO_FUNCIONAMENTO"].cast('string'))
escolas2020 = escolas2020.withColumn("DT_ANO_LETIVO_INICIO", escolas2020["DT_ANO_LETIVO_INICIO"].cast('string'))
escolas2020 = escolas2020.withColumn("DT_ANO_LETIVO_TERMINO", escolas2020["DT_ANO_LETIVO_TERMINO"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_REGIAO", escolas2020["CO_REGIAO"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_MESORREGIAO", escolas2020["CO_MESORREGIAO"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_MICRORREGIAO", escolas2020["CO_MICRORREGIAO"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_UF", escolas2020["CO_UF"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_MUNICIPIO", escolas2020["CO_MUNICIPIO"].cast('string'))
escolas2020 = escolas2020.withColumn("CO_DISTRITO", escolas2020["CO_DISTRITO"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_DEPENDENCIA", escolas2020["TP_DEPENDENCIA"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_LOCALIZACAO", escolas2020["TP_LOCALIZACAO"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_LOCALIZACAO_DIFERENCIADA", escolas2020["TP_LOCALIZACAO_DIFERENCIADA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_VINCULO_SECRETARIA_EDUCACAO", escolas2020["IN_VINCULO_SECRETARIA_EDUCACAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_VINCULO_SEGURANCA_PUBLICA", escolas2020["IN_VINCULO_SEGURANCA_PUBLICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_VINCULO_SECRETARIA_SAUDE", escolas2020["IN_VINCULO_SECRETARIA_SAUDE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_VINCULO_OUTRO_ORGAO", escolas2020["IN_VINCULO_OUTRO_ORGAO"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_CATEGORIA_ESCOLA_PRIVADA", escolas2020["TP_CATEGORIA_ESCOLA_PRIVADA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_CONVENIADA_PP", escolas2020["IN_CONVENIADA_PP"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_CONVENIO_PODER_PUBLICO", escolas2020["TP_CONVENIO_PODER_PUBLICO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_MANT_ESCOLA_PRIVADA_OSCIP", escolas2020["IN_MANT_ESCOLA_PRIVADA_OSCIP"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_MANT_ESCOLA_PRIV_ONG_OSCIP", escolas2020["IN_MANT_ESCOLA_PRIV_ONG_OSCIP"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_RESPONSAVEL_REGULAMENTACAO", escolas2020["TP_RESPONSAVEL_REGULAMENTACAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LOCAL_FUNC_SOCIOEDUCATIVO", escolas2020["IN_LOCAL_FUNC_SOCIOEDUCATIVO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LOCAL_FUNC_UNID_PRISIONAL", escolas2020["IN_LOCAL_FUNC_UNID_PRISIONAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LOCAL_FUNC_PRISIONAL_SOCIO", escolas2020["IN_LOCAL_FUNC_PRISIONAL_SOCIO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LOCAL_FUNC_GALPAO", escolas2020["IN_LOCAL_FUNC_GALPAO"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_OCUPACAO_GALPAO", escolas2020["TP_OCUPACAO_GALPAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LOCAL_FUNC_SALAS_OUTRA_ESC", escolas2020["IN_LOCAL_FUNC_SALAS_OUTRA_ESC"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LOCAL_FUNC_OUTROS", escolas2020["IN_LOCAL_FUNC_OUTROS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_PREDIO_COMPARTILHADO", escolas2020["IN_PREDIO_COMPARTILHADO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AGUA_POTAVEL", escolas2020["IN_AGUA_POTAVEL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AGUA_REDE_PUBLICA", escolas2020["IN_AGUA_REDE_PUBLICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AGUA_POCO_ARTESIANO", escolas2020["IN_AGUA_POCO_ARTESIANO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AGUA_CACIMBA", escolas2020["IN_AGUA_CACIMBA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AGUA_FONTE_RIO", escolas2020["IN_AGUA_FONTE_RIO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AGUA_INEXISTENTE", escolas2020["IN_AGUA_INEXISTENTE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ENERGIA_REDE_PUBLICA", escolas2020["IN_ENERGIA_REDE_PUBLICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ENERGIA_GERADOR_FOSSIL", escolas2020["IN_ENERGIA_GERADOR_FOSSIL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ENERGIA_RENOVAVEL", escolas2020["IN_ENERGIA_RENOVAVEL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ENERGIA_INEXISTENTE", escolas2020["IN_ENERGIA_INEXISTENTE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESGOTO_REDE_PUBLICA", escolas2020["IN_ESGOTO_REDE_PUBLICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESGOTO_FOSSA_SEPTICA", escolas2020["IN_ESGOTO_FOSSA_SEPTICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESGOTO_FOSSA_COMUM", escolas2020["IN_ESGOTO_FOSSA_COMUM"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESGOTO_FOSSA", escolas2020["IN_ESGOTO_FOSSA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESGOTO_INEXISTENTE", escolas2020["IN_ESGOTO_INEXISTENTE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LIXO_SERVICO_COLETA", escolas2020["IN_LIXO_SERVICO_COLETA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LIXO_QUEIMA", escolas2020["IN_LIXO_QUEIMA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LIXO_ENTERRA", escolas2020["IN_LIXO_ENTERRA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LIXO_DESTINO_FINAL_PUBLICO", escolas2020["IN_LIXO_DESTINO_FINAL_PUBLICO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LIXO_DESCARTA_OUTRA_AREA", escolas2020["IN_LIXO_DESCARTA_OUTRA_AREA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_TRATAMENTO_LIXO_SEPARACAO", escolas2020["IN_TRATAMENTO_LIXO_SEPARACAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_TRATAMENTO_LIXO_REUTILIZA", escolas2020["IN_TRATAMENTO_LIXO_REUTILIZA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_TRATAMENTO_LIXO_RECICLAGEM", escolas2020["IN_TRATAMENTO_LIXO_RECICLAGEM"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_TRATAMENTO_LIXO_INEXISTENTE", escolas2020["IN_TRATAMENTO_LIXO_INEXISTENTE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ALMOXARIFADO", escolas2020["IN_ALMOXARIFADO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AREA_VERDE", escolas2020["IN_AREA_VERDE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_AUDITORIO", escolas2020["IN_AUDITORIO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BANHEIRO", escolas2020["IN_BANHEIRO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BANHEIRO_EI", escolas2020["IN_BANHEIRO_EI"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BANHEIRO_PNE", escolas2020["IN_BANHEIRO_PNE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BANHEIRO_FUNCIONARIOS", escolas2020["IN_BANHEIRO_FUNCIONARIOS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BANHEIRO_CHUVEIRO", escolas2020["IN_BANHEIRO_CHUVEIRO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BIBLIOTECA", escolas2020["IN_BIBLIOTECA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BIBLIOTECA_SALA_LEITURA", escolas2020["IN_BIBLIOTECA_SALA_LEITURA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_COZINHA", escolas2020["IN_COZINHA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_DESPENSA", escolas2020["IN_DESPENSA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_DORMITORIO_ALUNO", escolas2020["IN_DORMITORIO_ALUNO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_DORMITORIO_PROFESSOR", escolas2020["IN_DORMITORIO_PROFESSOR"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LABORATORIO_CIENCIAS", escolas2020["IN_LABORATORIO_CIENCIAS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_LABORATORIO_INFORMATICA", escolas2020["IN_LABORATORIO_INFORMATICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_PATIO_COBERTO", escolas2020["IN_PATIO_COBERTO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_PATIO_DESCOBERTO", escolas2020["IN_PATIO_DESCOBERTO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_PARQUE_INFANTIL", escolas2020["IN_PARQUE_INFANTIL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_PISCINA", escolas2020["IN_PISCINA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_QUADRA_ESPORTES", escolas2020["IN_QUADRA_ESPORTES"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_QUADRA_ESPORTES_COBERTA", escolas2020["IN_QUADRA_ESPORTES_COBERTA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_QUADRA_ESPORTES_DESCOBERTA", escolas2020["IN_QUADRA_ESPORTES_DESCOBERTA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_REFEITORIO", escolas2020["IN_REFEITORIO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_ATELIE_ARTES", escolas2020["IN_SALA_ATELIE_ARTES"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_MUSICA_CORAL", escolas2020["IN_SALA_MUSICA_CORAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_ESTUDIO_DANCA", escolas2020["IN_SALA_ESTUDIO_DANCA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_MULTIUSO", escolas2020["IN_SALA_MULTIUSO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_DIRETORIA", escolas2020["IN_SALA_DIRETORIA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_LEITURA", escolas2020["IN_SALA_LEITURA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_PROFESSOR", escolas2020["IN_SALA_PROFESSOR"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_REPOUSO_ALUNO", escolas2020["IN_SALA_REPOUSO_ALUNO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SECRETARIA", escolas2020["IN_SECRETARIA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_SALA_ATENDIMENTO_ESPECIAL", escolas2020["IN_SALA_ATENDIMENTO_ESPECIAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_TERREIRAO", escolas2020["IN_TERREIRAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_VIVEIRO", escolas2020["IN_VIVEIRO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_DEPENDENCIAS_OUTRAS", escolas2020["IN_DEPENDENCIAS_OUTRAS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_CORRIMAO", escolas2020["IN_ACESSIBILIDADE_CORRIMAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_ELEVADOR", escolas2020["IN_ACESSIBILIDADE_ELEVADOR"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_PISOS_TATEIS", escolas2020["IN_ACESSIBILIDADE_PISOS_TATEIS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_VAO_LIVRE", escolas2020["IN_ACESSIBILIDADE_VAO_LIVRE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_RAMPAS", escolas2020["IN_ACESSIBILIDADE_RAMPAS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_SINAL_SONORO", escolas2020["IN_ACESSIBILIDADE_SINAL_SONORO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_SINAL_TATIL", escolas2020["IN_ACESSIBILIDADE_SINAL_TATIL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_SINAL_VISUAL", escolas2020["IN_ACESSIBILIDADE_SINAL_VISUAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSIBILIDADE_INEXISTENTE", escolas2020["IN_ACESSIBILIDADE_INEXISTENTE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_COMPUTADOR", escolas2020["IN_COMPUTADOR"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_COPIADORA", escolas2020["IN_EQUIP_COPIADORA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_IMPRESSORA", escolas2020["IN_EQUIP_IMPRESSORA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_IMPRESSORA_MULT", escolas2020["IN_EQUIP_IMPRESSORA_MULT"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_SCANNER", escolas2020["IN_EQUIP_DVD"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_SOM", escolas2020["IN_EQUIP_TV"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_LOUSA_DIGITAL", escolas2020["IN_EQUIP_LOUSA_DIGITAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_MULTIMIDIA", escolas2020["IN_EQUIP_MULTIMIDIA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_DESKTOP_ALUNO", escolas2020["IN_DESKTOP_ALUNO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_COMP_PORTATIL_ALUNO", escolas2020["IN_COMP_PORTATIL_ALUNO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_TABLET_ALUNO", escolas2020["IN_TABLET_ALUNO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_INTERNET", escolas2020["IN_INTERNET"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_INTERNET_ALUNOS", escolas2020["IN_INTERNET_ALUNOS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_INTERNET_ADMINISTRATIVO", escolas2020["IN_INTERNET_ADMINISTRATIVO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_INTERNET_APRENDIZAGEM", escolas2020["IN_INTERNET_APRENDIZAGEM"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_INTERNET_COMUNIDADE", escolas2020["IN_INTERNET_COMUNIDADE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACESSO_INTERNET_COMPUTADOR", escolas2020["IN_ACESSO_INTERNET_COMPUTADOR"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ACES_INTERNET_DISP_PESSOAIS", escolas2020["IN_ACES_INTERNET_DISP_PESSOAIS"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_REDE_LOCAL", escolas2020["TP_REDE_LOCAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_BANDA_LARGA", escolas2020["IN_BANDA_LARGA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ALIMENTACAO", escolas2020["IN_ALIMENTACAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EDUCACAO_INDIGENA", escolas2020["IN_EDUCACAO_INDIGENA"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_INDIGENA_LINGUA", escolas2020["TP_INDIGENA_LINGUA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EXAME_SELECAO", escolas2020["IN_EXAME_SELECAO"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_REDES_SOCIAIS", escolas2020["IN_REDES_SOCIAIS"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESPACO_ATIVIDADE", escolas2020["IN_ESPACO_ATIVIDADE"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESPACO_EQUIPAMENTO", escolas2020["IN_ESPACO_EQUIPAMENTO"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_PROPOSTA_PEDAGOGICA", escolas2020["TP_PROPOSTA_PEDAGOGICA"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_AEE", escolas2020["TP_AEE"].cast('string'))
escolas2020 = escolas2020.withColumn("TP_ATIVIDADE_COMPLEMENTAR", escolas2020["TP_ATIVIDADE_COMPLEMENTAR"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_MEDIACAO_PRESENCIAL", escolas2020["IN_MEDIACAO_PRESENCIAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_MEDIACAO_SEMIPRESENCIAL", escolas2020["IN_MEDIACAO_SEMIPRESENCIAL"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_MEDIACAO_EAD", escolas2020["IN_MEDIACAO_EAD"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_ESPECIAL_EXCLUSIVA", escolas2020["IN_ESPECIAL_EXCLUSIVA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_PARABOLICA", escolas2020["IN_EQUIP_PARABOLICA"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_NENHUM", escolas2020["IN_EQUIP_NENHUM"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_DVD", escolas2020["IN_EQUIP_DVD"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_EQUIP_TV", escolas2020["IN_EQUIP_TV"].cast('string'))
escolas2020 = escolas2020.withColumn("IN_MATERIAL_PED_NENHUM", escolas2020["IN_MATERIAL_PED_NENHUM"].cast('string'))

In [117]:
# Transforma as colunas de quantidade para INT
escolas2020 = escolas2020.withColumn("QT_SALAS_UTILIZADAS_DENTRO", escolas2020["QT_SALAS_UTILIZADAS_DENTRO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_SALAS_UTILIZADAS_FORA", escolas2020["QT_SALAS_UTILIZADAS_FORA"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_SALAS_UTILIZADAS", escolas2020["QT_SALAS_UTILIZADAS"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_SALAS_UTILIZA_CLIMATIZADAS", escolas2020["QT_SALAS_UTILIZA_CLIMATIZADAS"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_SALAS_UTILIZADAS_ACESSIVEIS", escolas2020["QT_SALAS_UTILIZADAS_ACESSIVEIS"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_EQUIP_DVD", escolas2020["QT_EQUIP_DVD"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_EQUIP_SOM", escolas2020["QT_EQUIP_SOM"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_EQUIP_TV", escolas2020["QT_EQUIP_TV"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_EQUIP_MULTIMIDIA", escolas2020["QT_EQUIP_MULTIMIDIA"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_EQUIP_LOUSA_DIGITAL", escolas2020["QT_EQUIP_LOUSA_DIGITAL"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_DESKTOP_ALUNO", escolas2020["QT_DESKTOP_ALUNO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_COMP_PORTATIL_ALUNO", escolas2020["QT_COMP_PORTATIL_ALUNO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_TABLET_ALUNO", escolas2020["QT_TABLET_ALUNO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_ADMINISTRATIVOS", escolas2020["QT_PROF_ADMINISTRATIVOS"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_SERVICOS_GERAIS", escolas2020["QT_PROF_SERVICOS_GERAIS"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_BIBLIOTECARIO", escolas2020["QT_PROF_BIBLIOTECARIO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_SAUDE", escolas2020["QT_PROF_SAUDE"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_COORDENADOR", escolas2020["QT_PROF_COORDENADOR"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_FONAUDIOLOGO", escolas2020["QT_PROF_FONAUDIOLOGO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_NUTRICIONISTA", escolas2020["QT_PROF_NUTRICIONISTA"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_PSICOLOGO", escolas2020["QT_PROF_PSICOLOGO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_ALIMENTACAO", escolas2020["QT_PROF_ALIMENTACAO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_PEDAGOGIA", escolas2020["QT_PROF_PEDAGOGIA"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_SECRETARIO", escolas2020["QT_PROF_SECRETARIO"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_SEGURANCA", escolas2020["QT_PROF_SEGURANCA"].cast('int'))
escolas2020 = escolas2020.withColumn("QT_PROF_MONITORES", escolas2020["QT_PROF_MONITORES"].cast('int'))

In [118]:
# Salva os dados das ESCOLAS 2019 na pasta OURO do bucket como arquivo PARQUET
escolas2020.write.parquet('gs://igti-pa/ouro/escolas2020.parquet')

21/12/13 20:50:26 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [93]:
##### TESTE DE INTEGRIDADE DOS DADOS - CENSO 2020

In [94]:
spark.read.parquet('gs://igti-pa/ouro/docentes2020.parquet')

DataFrame[DATA_NASCIMENTO: string, NU_IDADE_REFERENCIA: string, TP_SEXO: string, TP_COR_RACA: string, CO_UF_NASC: string, CO_MUNICIPIO_NASC: string, CO_UF_END: string, CO_MUNICIPIO_END: string, TP_ZONA_RESIDENCIAL: string, TP_LOCAL_RESID_DIFERENCIADA: string, IN_NECESSIDADE_ESPECIAL: string, TP_ESCOLARIDADE: string, TP_ENSINO_MEDIO: string, IN_COMPLEMENTACAO_PEDAGOGICA: string, IN_ESPECIALIZACAO: string, IN_MESTRADO: string, IN_DOUTORADO: string, IN_POS_NENHUM: string, IN_ESPECIFICO_GESTAO: string, ID_TURMA: string, TP_TIPO_DOCENTE: string, TP_TIPO_CONTRATACAO: string, TP_ETAPA_ENSINO: string, IN_ESPECIAL_EXCLUSIVA: string, CO_ENTIDADE: string]

In [95]:
spark.read.parquet('gs://igti-pa/ouro/matriculas2020.parquet')

DataFrame[ID_MATRICULA: string, DATA_NASCIMENTO: string, NU_IDADE_REFERENCIA: string, TP_SEXO: string, TP_COR_RACA: string, CO_UF_NASC: string, CO_MUNICIPIO_NASC: string, CO_UF_END: string, CO_MUNICIPIO_END: string, TP_ZONA_RESIDENCIAL: string, IN_NECESSIDADE_ESPECIAL: string, TP_ETAPA_ENSINO: string, IN_ESPECIAL_EXCLUSIVA: string, ID_TURMA: string, NU_DIAS_ATIVIDADE: string, TP_TIPO_ATENDIMENTO_TURMA: string, TP_TIPO_LOCAL_TURMA: string, CO_ENTIDADE: string]

In [96]:
spark.read.parquet('gs://igti-pa/ouro/turmas2020.parquet')

DataFrame[ID_TURMA: string, TP_LOCALIZACAO: string, TP_ETAPA_ENSINO: string, QT_MATRICULAS: int, NU_DIAS_ATIVIDADE: string, TP_MEDIACAO_DIDATICO_PEDAGO: string, TP_TIPO_ATENDIMENTO_TURMA: string, TP_TIPO_LOCAL_TURMA: string, IN_ESPECIAL_EXCLUSIVA: string, CO_ENTIDADE: string]

In [97]:
spark.read.parquet('gs://igti-pa/ouro/escolas2020.parquet')

DataFrame[CO_ENTIDADE: string, NO_ENTIDADE: string, CO_ORGAO_REGIONAL: string, TP_SITUACAO_FUNCIONAMENTO: string, DT_ANO_LETIVO_INICIO: string, DT_ANO_LETIVO_TERMINO: string, CO_REGIAO: string, CO_MESORREGIAO: string, CO_MICRORREGIAO: string, CO_UF: string, CO_MUNICIPIO: string, CO_DISTRITO: string, TP_DEPENDENCIA: string, TP_LOCALIZACAO: string, TP_LOCALIZACAO_DIFERENCIADA: string, IN_VINCULO_SECRETARIA_EDUCACAO: string, IN_VINCULO_SEGURANCA_PUBLICA: string, IN_VINCULO_SECRETARIA_SAUDE: string, IN_VINCULO_OUTRO_ORGAO: string, TP_CATEGORIA_ESCOLA_PRIVADA: string, IN_CONVENIADA_PP: string, TP_CONVENIO_PODER_PUBLICO: string, IN_MANT_ESCOLA_PRIVADA_OSCIP: string, IN_MANT_ESCOLA_PRIV_ONG_OSCIP: string, TP_RESPONSAVEL_REGULAMENTACAO: string, IN_LOCAL_FUNC_SOCIOEDUCATIVO: string, IN_LOCAL_FUNC_UNID_PRISIONAL: string, IN_LOCAL_FUNC_PRISIONAL_SOCIO: string, IN_LOCAL_FUNC_GALPAO: string, TP_OCUPACAO_GALPAO: string, IN_LOCAL_FUNC_SALAS_OUTRA_ESC: string, IN_LOCAL_FUNC_OUTROS: string, IN_PREDIO_C

### FIM DO SCRIPT ETL3